<a href="https://colab.research.google.com/github/vishnubob/AoC/blob/main/AoC_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import sys
import math
import collections
import functools
import itertools
import operator
import requests
import numpy as np
import scipy.signal
from tqdm import tqdm
from pprint import pprint

np.set_printoptions(threshold=sys.maxsize, linewidth=np.inf)

cache_dir = "/content/drive/MyDrive/AoC2022/"

session_fn = f"{cache_dir}/session_id"
with open(session_fn) as fh:
    session_id = fh.read().strip()

def aoc_input(day, split=True):
    url = f"https://adventofcode.com/2022/day/{day}/input"
    cache_fn = f"{cache_dir}/d{day}.dat"
    if not os.path.exists(cache_fn):
        cookies = {'session': session_id}
        resp = requests.get(url, cookies=cookies)
        with open(cache_fn, 'wb') as fh:
            fh.write(resp.content)
    with open(cache_fn) as fh:
        data = fh.read()
    data = data.strip()
    if split:
        data = data.split('\n')
    return data

In [2]:
inp = aoc_input(1)

def parse_input(inp):
    vals = []
    for item in inp:
        if not item:
            yield tuple(vals)
            vals = []
            continue
        vals.append(int(item))
    if vals:
        yield tuple(vals)

# d1p1
sums = list(map(sum, parse_input(inp)))
print(max(sums))
# d1p2
print(sum(sorted(sums, reverse=True)[:3]))


68775
202585


In [3]:
inp = aoc_input(2)

def win_lose_draw(game):
    (x, y) = game
    if ((y + 1) % 3) == x:
        return (y + 1)
    if ((y - 1) % 3) == x:
        return (y + 1 + 6)
    return (y + 1 + 3)

def next_move(game):
    (x, y) = game
    if y == 0:
        return (x, (x - 1) % 3)
    if y == 1:
        return (x, x)
    return (x, (x + 1) % 3)

nums = lambda it: it.split(' ')
inp = [(2 - (ord('C') - ord(x)), 2 - (ord('Z') - ord(y))) for (x, y) in map(nums, inp)]
print(sum(map(win_lose_draw, inp)))
print(sum(map(win_lose_draw, map(next_move, inp))))


10624
14060


In [4]:
inp = aoc_input(3)

packs = [(set(val[:len(val) // 2]) & set(val[len(val) // 2:])) for val in inp]
maps = {chr(ord('a') + x): (x + 1) for x in range(26)}
maps.update({chr(ord('A') + x): (x + 27) for x in range(26)})
print(sum([maps[it[0]] for it in map(list, packs)]))

i = iter(inp)
print(sum([maps[list(it)[0]] for it in [set.intersection(*map(set, it)) for it in (zip(i, i, i))]]))


7428
2650


In [5]:
inp = aoc_input(4)

to_intv = lambda it: tuple(map(int, it.split('-')))
data = [tuple(map(to_intv, line.split(','))) for line in inp]
is_contained = lambda it: (it[0][0] <= it[1][0] and it[0][1] >= it[1][1]) or (it[0][0] >= it[1][0] and it[0][1] <= it[1][1])
print(sum([is_contained(it) for it in data]))
is_overlap = lambda it: (it[0][0] >= it[1][0] and it[0][0] <= it[1][1]) or (it[0][1] >= it[1][0] and it[0][1] <= it[1][1]) or is_contained(it)
print(sum([is_overlap(it) for it in data]))


556
876


In [6]:
inp = aoc_input(5)
space_re = re.compile('\s+')
crate_re = re.compile('[\w]')

moves = []
crates = []
for line in inp:
    if not line.strip():
        continue
    if 'move' in line:
        line = line.split(' ')
        moves.append((int(line[1]), int(line[3]), int(line[5])))
        continue
    if line.startswith(' 1'):
        continue
    crates.append(line[1::4])

max_len = max([len(ln) for ln in crates])
new_crates = []
for ln in crates:
    n_pad = max_len - len(ln)
    ln += ' ' * n_pad
    new_crates.append(ln)
crate_init = new_crates

crates = [list(filter(bool, map(str.strip, st)))[::-1] for st in list(zip(*crate_init))]
for (cnt, from_stack, to_stack) in moves:
    payload = crates[from_stack - 1][-cnt:]
    crates[to_stack - 1] += payload[::-1]
    crates[from_stack - 1] = crates[from_stack - 1][:-cnt]

tops = str.join('', crate_re.findall(str.join('', [it[-1] for it in crates])))
print(tops)

crates = [list(filter(bool, map(str.strip, st)))[::-1] for st in list(zip(*crate_init))]
for (cnt, from_stack, to_stack) in moves:
    payload = crates[from_stack - 1][-cnt:]
    crates[to_stack - 1] += payload
    crates[from_stack - 1] = crates[from_stack - 1][:-cnt]

tops = str.join('', crate_re.findall(str.join('', [it[-1] for it in crates])))
print(tops)

TBVFVDZPN
VLCWHTDSZ


In [7]:
inp = aoc_input(6)

for line in inp:
    if not line.strip():
        continue
    kmers = [(x + 4, len(set(line[x:x+4])) == 4) for x in range(len(line) - 4)]
    for (idx, val) in kmers:
        if val:
            print(idx)
            break

for line in inp:
    if not line.strip():
        continue
    kmers = [(x + 14, len(set(line[x:x+14])) == 14) for x in range(len(line) - 14)]
    for (idx, val) in kmers:
        if val:
            print(idx)
            break

1287
3716


In [8]:
inp = aoc_input(7)

class Directory:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.files = {}
        self.dirs = {}
    
    def add_file(self, filename=None, size=None):
        self.files[filename] = size

    def add_directory(self, name=None):
        self.dirs[name] = self.__class__(name=name, parent=self)
    
    def get_directory(self, name=None):
        if name not in self.dirs:
            self.add_directory(name=name)
        return self.dirs[name]
    
    @property
    def local_size(self):
        return sum(self.files.values())

    @property
    def size(self):
        size = 0
        for child in self.dirs.values():
            size += child.size
        return size + self.local_size
    
    def flatten(self):
        for child in self.dirs.values():
            yield from child.flatten()
        yield self

root = Directory(name='/', parent=None)
fs = root

for line in inp:
    parts = line.split(' ')
    if parts[0] == '$':
        if parts[1] == 'cd':
            dir_name = parts[-1]
            if dir_name == '..':
                fs = fs.parent
            else:
                fs = fs.get_directory(dir_name)
        continue
    elif parts[0] == 'dir':
        dir_name = parts[-1]
        fs.add_directory(dir_name)
    else:
        size = int(parts[0])
        filename = parts[1]
        fs.add_file(filename=filename, size=size)

sizes = [node.size for node in root.flatten()]
print(sum([sz for sz in sizes if sz < 1e5]))

total_size = int(7e7)
required_capacity = int(3e7)
available_capacity = total_size - root.size
needed_capacity = required_capacity - available_capacity
deltas = [(node.name, node.size - needed_capacity, node.size) for node in root.flatten() if node.size - needed_capacity > 0]  
deltas = sorted(deltas, key=lambda it: it[1])
print(deltas[0][-1])

1297683
5756764


In [9]:
inp = aoc_input(8)
inp = np.array([list(map(int, line)) for line in inp])

vis_count = (inp.shape[0] - 2) * 4 + 4
for x in range(1, inp.shape[0] - 1):
    for y in range(1, inp.shape[1] - 1):
        val = inp[x][y]
        row = 1 * (inp[x, :] < val)
        col = 1 * (inp[:, y] < val)
        codes = (row[:y], row[y+1:], col[:x], col[x+1:])
        vis_count += int(any([np.sum(code) == len(code) for code in codes]))
print(vis_count)

scores = np.zeros_like(inp)
for x in range(1, inp.shape[0] - 1):
    for y in range(1, inp.shape[1] - 1):
        val = inp[x][y]
        row = 1 * (inp[x, :] < val)
        col = 1 * (inp[:, y] < val)
        codes = (row[:y][::-1], row[y+1:], col[:x][::-1], col[x+1:])
        score = 1
        for code in codes:
            length = 0
            for val in code:
                length += 1
                if not val:
                    break
            score *= length
        scores[x][y] = score

print(np.max(scores))

1827
335580


In [10]:
inp = aoc_input(9)

DirectionMap = {
    'R': np.array([0, 1]),
    'L': np.array([0, -1]),
    'D': np.array([-1, 0]),
    'U': np.array([1, 0]),
}

def run_sim(move_list=None, knot_count=2, pos=(0, 0)):
    move_hash = set()
    knots = np.ones((knot_count, 2), dtype=int) * list(pos)
    move_hash.add(tuple(pos))
    for (direction, count) in move_list:
        vec = DirectionMap[direction]
        for step in range(count):
            knots[0, :] += vec
            for knot_idx in range(1, knot_count):
                delta = knots[knot_idx - 1, :] - knots[knot_idx, :]
                mag = np.linalg.norm(delta)
                if mag < 2:
                    break
                knots[knot_idx, :] += np.sign(delta)
            move_hash.add(tuple(knots[-1, :]))
    return len(move_hash)

move_list = list(map(lambda it: (it[0], int(it[1])), [it.split(' ') for it in inp]))
print(run_sim(move_list=move_list, knot_count=2))
print(run_sim(move_list=move_list, knot_count=10))

5513
2427


In [11]:
inp = aoc_input(10)

periods = [20, 60, 100, 140, 180, 220]
ary = {}
x = 1
next_x = None
cycle = 1
cols = 40
rows = 6

for line in inp:
    parts = line.split(' ')
    if next_x is not None:
        x = next_x
        next_x = None
    if len(parts) == 1:
        ary[cycle] = x
        cycle += 1
    elif len(parts) == 2:
        next_x = x + int(parts[-1])
        ary[cycle] = x
        cycle += 1
        ary[cycle] = x
        cycle += 1

print(sum([pt * ary[pt] for pt in periods]))

fb = np.zeros((6*40), dtype=int)
for cycle in range(rows * cols):
    sprite = np.zeros((40,), dtype=int)
    x = ary[cycle+1]
    sprite[x-1:x+2] = 1
    if sprite[cycle % 40]:
        fb[cycle] = 1

for line in fb.reshape((rows, cols)):
    line = str.join('', map(str, list(line)))
    line = line.replace('0', ' ')
    line = line.replace('1', '#')
    print(line)


11720
#### ###   ##  ###  #### ###   ##    ## 
     #  # #  # #  # #    #  # #  #    # 
###  #  # #    #  # ###  #  # #       # 
     ###  #    ###  #    ###  #       # 
#    # #  #  # # #  #    #    #  # #  # 
#### #  #  ##  #  # #### #     ##   ##  


In [12]:
class Monkey:
    def __init__(self, idx=None, items=None, operation=None, phase=None, neighbors=None):
        self.idx = idx
        self.items = items
        self.operation = operation.split(' ')
        self.phase = phase
        self.neighbors = neighbors
        self.inspected = 0
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}(idx={self.idx}, items={self.items}, operation={repr(self.operation)}, phase={self.phase}, neighbors={self.neighbors})'
        return rep
    
    @classmethod
    def parse(cls, txt):
        lines = txt.split('\n')
        assert lines[0].startswith('Monkey')
        monkey_idx = int(lines[0].split(' ')[-1][:-1])
        items = list(map(int, lines[1].split(':')[-1].split(',')))
        operation = lines[2].split('=')[1][1:]
        phase = int(lines[3].split(' ')[-1])
        neighbors = [int(lines[4].split(' ')[-1]),  int(lines[5].split(' ')[-1])]
        monkey_spec = dict(idx=monkey_idx, items=items, operation=operation, phase=phase, neighbors=neighbors)
        return cls(**monkey_spec)
    
    def do_monkey_business(self, monkey_list=None, relax=3, shared_key=None):
        items = self.items[:]
        self.items = list()
        for item in items:
            if self.operation[1] == '+':
                item = item + int(self.operation[2])
            elif self.operation[1] == '*':
                if self.operation[2] == 'old':
                    item = item ** 2
                else:
                    item = item * int(self.operation[2])
            else:
                raise(0)
            if relax > 1:
                item = int(math.floor(item / relax))
            if shared_key is not None:
                item = item % shared_key

            div_idx = int((item % self.phase) != 0)
            nn = self.neighbors[div_idx]
            monkey_list[nn].items.append(item)
        self.inspected += len(items)

inp = aoc_input(11, split=False)
inp = inp.split('\n\n')

monkey_list = [Monkey.parse(monkey_spec) for monkey_spec in inp]

for n_round in range(20):
    for monkey in monkey_list:
        monkey.do_monkey_business(monkey_list=monkey_list)

(monk_a, monk_b) = sorted([it.inspected for it in monkey_list])[-2:]
print(monk_a * monk_b)

monkey_list = [Monkey.parse(monkey_spec) for monkey_spec in inp]
shared_key = functools.reduce(operator.mul, [monkey.phase for monkey in monkey_list], 1)
for n_round in range(10000):
    for monkey in monkey_list:
        monkey.do_monkey_business(monkey_list=monkey_list, relax=1, shared_key=shared_key)

(monk_a, monk_b) = sorted([it.inspected for it in monkey_list])[-2:]
print(monk_a * monk_b)

119715
18085004878


In [13]:
def dfs(grid, start_pos, stop_pos):
    start_pos = tuple(start_pos)
    que = [start_pos]
    visited = np.ones_like(grid) * -1
    visited[start_pos] = 0
    nn_vec =  [np.array([0, 1]), np.array([0, -1]),  np.array([-1, 0]),  np.array([1, 0])]

    while que:
        node = que.pop()
        level = visited[node] + 1
        for vec in nn_vec:
            idx = np.array(node + vec)
            bb_min = np.array([0, 0])
            bb_max = np.array(grid.shape) - 1
            idx = np.max((bb_min, idx), axis=0)
            idx = np.min((bb_max, idx), axis=0)
            idx = tuple(idx)
            if visited[idx] >= 0:
                continue
            if (grid[idx] - grid[node]) > 1:
                continue
            visited[idx] = level
            que = [idx] + que
            if idx == stop_pos:
                return level

inp = aoc_input(12)
inp = [list(map(lambda ch: ord(ch) - ord('a'), line)) for line in inp]
grid = np.array(inp, dtype=int)
grid[grid == (ord('S') - ord('a'))] = -1
grid[grid == (ord('E') - ord('a'))] = np.max(grid) + 1
start_pos = tuple(np.argwhere(grid == -1)[0])
stop_pos = tuple(np.argwhere(grid == 26)[0])
print(dfs(grid, start_pos, stop_pos))
levels = [dfs(grid, start_pos, stop_pos) for start_pos in np.argwhere(grid == 0)]
levels = [level for level in levels if level is not None]
print(min(levels))


352
345
